## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Concordia,AR,-31.39,-58.02,37.00,82,0,4.92,clear sky
1,1,Arman,RU,59.70,150.17,55.71,46,92,13.60,overcast clouds
2,2,Busselton,AU,-33.65,115.33,66.99,49,95,12.12,overcast clouds
3,3,Banjar,ID,-8.19,114.97,84.20,89,40,16.11,scattered clouds
4,4,Kirakira,SB,-10.45,161.92,84.49,75,91,10.67,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you are looking for on your trip? "))
max_temp = float(input("What is the maximum temperature you are looking for on your trip? "))

What is the minimum temperature you are looking for on your trip? 65
What is the maximum temperature you are looking for on your trip? 75


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Busselton,AU,-33.65,115.33,66.99,49,95,12.12,overcast clouds
7,7,San Cristobal,VE,7.77,-72.22,67.28,92,100,1.86,moderate rain
9,9,Vila Velha,BR,-20.33,-40.29,68.00,72,75,11.41,broken clouds
12,12,Albany,US,42.60,-73.97,70.00,88,89,2.84,overcast clouds
13,13,Mahebourg,MU,-20.41,57.70,73.40,64,20,8.05,few clouds
22,22,Passo De Camaragibe,BR,-9.24,-35.49,68.00,100,38,3.36,scattered clouds
25,25,Acushnet,US,41.68,-70.91,72.00,83,1,8.05,clear sky
29,29,Rikitea,PF,-23.12,-134.97,65.75,71,41,21.09,scattered clouds
37,37,Largu,RO,44.97,27.15,69.80,64,5,3.36,clear sky
47,47,Ponta Do Sol,PT,32.67,-17.10,73.40,83,75,10.29,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                171
City                   171
Country                170
Lat                    171
Lng                    171
Max Temp               171
Humidity               171
Cloudiness             171
Wind Speed             171
Current Description    171
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_final_df = preferred_cities_df.dropna()
preferred_cities_final_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Busselton,AU,-33.65,115.33,66.99,49,95,12.12,overcast clouds
7,7,San Cristobal,VE,7.77,-72.22,67.28,92,100,1.86,moderate rain
9,9,Vila Velha,BR,-20.33,-40.29,68.00,72,75,11.41,broken clouds
12,12,Albany,US,42.60,-73.97,70.00,88,89,2.84,overcast clouds
13,13,Mahebourg,MU,-20.41,57.70,73.40,64,20,8.05,few clouds
...,...,...,...,...,...,...,...,...,...,...
711,711,Fairbanks,US,64.84,-147.72,69.80,64,40,3.36,scattered clouds
712,712,Bose,IT,45.46,8.01,75.00,78,11,3.00,few clouds
715,715,Portoferraio,IT,42.81,10.31,68.00,10,0,10.33,clear sky
716,716,Grand Baie,MU,-20.02,57.58,72.00,72,1,5.01,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_final_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Busselton,AU,66.99,overcast clouds,-33.65,115.33,
7,San Cristobal,VE,67.28,moderate rain,7.77,-72.22,
9,Vila Velha,BR,68.00,broken clouds,-20.33,-40.29,
12,Albany,US,70.00,overcast clouds,42.60,-73.97,
13,Mahebourg,MU,73.40,few clouds,-20.41,57.70,
22,Passo De Camaragibe,BR,68.00,scattered clouds,-9.24,-35.49,
25,Acushnet,US,72.00,clear sky,41.68,-70.91,
29,Rikitea,PF,65.75,scattered clouds,-23.12,-134.97,
37,Largu,RO,69.80,clear sky,44.97,27.15,
47,Ponta Do Sol,PT,73.40,broken clouds,32.67,-17.10,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]    
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Busselton,AU,66.99,overcast clouds,-33.65,115.33,Observatory Guest House
7,San Cristobal,VE,67.28,moderate rain,7.77,-72.22,Pirineos
9,Vila Velha,BR,68.00,broken clouds,-20.33,-40.29,Hotel Vitória Palace
12,Albany,US,70.00,overcast clouds,42.60,-73.97,NaN
13,Mahebourg,MU,73.40,few clouds,-20.41,57.70,Le Peninsula Bay Beach Resort & Spa


In [20]:
# 7. Drop the rows where there is no Hotel Name.
# 7a. Chacking for any null Hotel Names
hotel_df.count()

City                   170
Country                170
Max Temp               170
Current Description    170
Lat                    170
Lng                    170
Hotel Name             149
dtype: int64

In [21]:
# 7b. Removing Hotels with null values
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Busselton,AU,66.99,overcast clouds,-33.65,115.33,Observatory Guest House
7,San Cristobal,VE,67.28,moderate rain,7.77,-72.22,Pirineos
9,Vila Velha,BR,68.00,broken clouds,-20.33,-40.29,Hotel Vitória Palace
13,Mahebourg,MU,73.40,few clouds,-20.41,57.70,Le Peninsula Bay Beach Resort & Spa
22,Passo De Camaragibe,BR,68.00,scattered clouds,-9.24,-35.49,Pousada Divino Milagre
25,Acushnet,US,72.00,clear sky,41.68,-70.91,Hampton Inn New Bedford/Fairhaven
29,Rikitea,PF,65.75,scattered clouds,-23.12,-134.97,Pension Maro'i
47,Ponta Do Sol,PT,73.40,broken clouds,32.67,-17.10,Hotel do Campo
51,San Carlos,VE,73.09,overcast clouds,9.66,-68.58,Hotel Paternopoli
53,Khani,GE,68.00,light intensity shower rain,41.96,42.96,კურორტი ზეკარი/Resort Zekari


In [22]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))